In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import sklearn.metrics as mt
from sklearn.model_selection import StratifiedShuffleSplit
import sklearn.preprocessing as pre

In [2]:
table = pd.read_csv('database.csv',na_values=['NA','NaN','.','?',' ','#VALUE!']).dropna(subset=['Act Fis']).fillna(0)
X = pre.scale(table.drop(columns=['Act Fis','Muestra','FICHA','ESC','Procedencia']).to_numpy().astype(float))
y = table['Act Fis'].to_numpy()

le = pre.LabelEncoder()
le.fit(y)
y = le.transform(y)

In [3]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2)
for train_index, test_index in sss.split(X,y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

In [4]:
param_nn = {'alpha': [0.0001], 'hidden_layer_sizes': [(200,100,50,)], 'max_iter':[10000]}
nn = MLPClassifier()
cv_nn = GridSearchCV(nn,param_nn)
cv_nn.fit(X_train,y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=MLPClassifier(activation='relu', alpha=0.0001,
                                     batch_size='auto', beta_1=0.9,
                                     beta_2=0.999, early_stopping=False,
                                     epsilon=1e-08, hidden_layer_sizes=(100,),
                                     learning_rate='constant',
                                     learning_rate_init=0.001, max_fun=15000,
                                     max_iter=200, momentum=0.9,
                                     n_iter_no_change=10,
                                     nesterovs_momentum=True, power_t=0.5,
                                     random_state=None, shuffle=True,
                                     solver='adam', tol=0.0001,
                                     validation_fraction=0.1, verbose=False,
                                     warm_start=False),
             iid='deprecated', n_jobs=None,
             param_gr

In [7]:
predicted_nn = cv_nn.predict(X_test)
accuracy_nn = mt.accuracy_score(y_test,predicted_nn)
print(accuracy_nn)
precision = mt.precision_score(y_test,predicted_nn)
print(precision)

1.0
1.0


array([[-1.01052952, -0.04148806, -0.04989447, ...,  1.49914505,
         1.09851915,  1.71711716],
       [-1.01052952, -0.02674   , -0.04989447, ...,  1.49914505,
         0.8844739 ,  1.82374043],
       [-1.01052952,  0.26822116,  0.24348823, ...,  1.41165755,
         1.66930649,  1.46959883],
       ...,
       [ 0.98958019,  1.10886048,  1.12363631, ..., -0.62555151,
        -0.59798767, -0.62859493],
       [ 0.98958019,  1.25634107,  1.27032766, ..., -0.62555151,
        -0.59798767, -0.62859493],
       [ 0.98958019,  1.69878281,  1.7104017 , ..., -0.62555151,
        -0.59798767, -0.62859493]])

In [13]:
pos = 0
neg = 0
for i in range(y_test.shape[0]):
    if y_test[i] == 1:
        pos += 1
    elif y_test[i] == 0:
        neg += 1
        
print(pos)
print(neg)

450
276
